In [ ]:
def iv_object(df, feature):
    """
    Set pr=True to enable printing of output.
    
    Output: 
      * iv: float,
      * woetable: pandas.woetableFrame
    """

    lst = []
    for i in range(df[feature].nunique()):
        val = list(df[feature].unique())[i]
        lst.append([feature,
                    'None',# Variable
                    'None',
                    val,  
                    'None',
                    'None',
                    'None',# Value
                    df[df[feature] == val].count()[feature],                        # All
                    df[(df[feature] == val) & (df['bg'] == 0)].count()[feature],  # Good (think: Fraud == 0)
                    df[(df[feature] == val) & (df['bg'] == 1)].count()[feature]]) # Bad (think: Fraud == 1)

    woetable = pd.DataFrame(lst, columns=['objVariable1','objVariable2','numVariable','objValue1','objValue2', 'numValue1','numValue2', 'All', 'Good', 'Bad'])

    woetable['Share'] = woetable['All'] / woetable['All'].sum()
    woetable['Bad Rate'] = woetable['Bad'] / woetable['All']
    woetable['Distribution Good'] = (woetable['All'] - woetable['Bad']) / (woetable['All'].sum() - woetable['Bad'].sum())
    woetable['Distribution Bad'] = woetable['Bad'] / woetable['Bad'].sum()
    woetable['WoE'] = np.log(woetable['Distribution Good'] / woetable['Distribution Bad'])

    woetable = woetable.replace({'WoE': {np.inf: 0, -np.inf: 0}})

    woetable['IV'] = woetable['WoE'] * (woetable['Distribution Good'] - woetable['Distribution Bad'])

    woetable = woetable.sort_values(by=['objVariable1', 'objValue1'], ascending=[True,True])
    woetable.index = range(len(woetable.index))
    woetable = woetable.dropna()

    iv = woetable['IV'].sum()
    # print(iv)

    return woetable#.values.tolist()

def iv_object_object(df, feature1,feature2):
    """
    Set pr=True to enable printing of output.
    
    Output: 
      * iv: float,
      * woetable: pandas.woetableFrame
    """

    lst = []
    for m in range(df[feature1].nunique()):
        val1 = list(df[feature1].unique())[m]
        for i in range(df[feature2].nunique()):
            val2 = list(df[feature2].unique())[i]
            lst.append([feature1,   
                        feature2, # Variable
                        'None',
                        val1,
                        val2,# Value
                        'None',
                        'None',
                        df[(df[feature1] == val1) &(df[feature2] == val2)].count()[feature2],                        # All
                        df[(df[feature1] == val1) &(df[feature2] == val2) & (df['bg'] == 0)].count()[feature2],  # Good (think: Fraud == 0)
                        df[(df[feature1] == val1) &(df[feature2] == val2) & (df['bg'] == 1)].count()[feature2]]) # Bad (think: Fraud == 1)

    woetable = pd.DataFrame(lst, columns=['objVariable1','objVariable2','numVariable','objValue1','objValue2', 'numValue1','numValue2', 'All', 'Good', 'Bad'])

    woetable['Share'] = woetable['All'] / woetable['All'].sum()
    woetable['Bad Rate'] = woetable['Bad'] / woetable['All']
    woetable['Distribution Good'] = (woetable['All'] - woetable['Bad']) / (woetable['All'].sum() - woetable['Bad'].sum())
    woetable['Distribution Bad'] = woetable['Bad'] / woetable['Bad'].sum()
    woetable['WoE'] = np.log(woetable['Distribution Good'] / woetable['Distribution Bad'])

    woetable = woetable.replace({'WoE': {np.inf: 0, -np.inf: 0}})

    woetable['IV'] = woetable['WoE'] * (woetable['Distribution Good'] - woetable['Distribution Bad'])

    woetable = woetable.sort_values(by=['objVariable1','objVariable2','objValue1','objValue2'], ascending=[True,True,True,True])
    woetable.index = range(len(woetable.index))
    woetable = woetable.dropna()

    iv = woetable['IV'].sum()
    # print(iv)

    return woetable#.values.tolist()

def iv_numeric(df, feature,num=20,scale=None):
    """
    Set pr=True to enable printing of output.
    
    Output: 
      * iv: float,
      * woetable: pandas.woetableFrame
    """
    iv=0
    lst = []
    h=100/np.arange(2,num)
    if(scale is None):
        for k in h:
            lst = []
            for i in np.arange(0,100//k):
                val1=-99999 if i==np.arange(0,100//k)[0] else val2# np.percentile(df[feature],k*i)
                val2=999999 if i==np.arange(0,100//k)[len(np.arange(0,100//k))-1] else np.percentile(df[feature],k*(i+1))
                lst.append(['None',
                            'None',
                            feature,
                            'None',# Variable
                            'None',
                            val1,                                                            # Value1
                            val2,
                            df[(df[feature]>=val1) & (df[feature] < val2)].count()[feature],                        # All
                            df[(df[feature]>=val1) & (df[feature] < val2) & (df['bg'] == 0)].count()[feature],  # Good (think: Fraud == 0)
                            df[(df[feature]>=val1) & (df[feature] < val2) & (df['bg'] == 1)].count()[feature]]) # Bad (think: Fraud == 1)

            woetable = pd.DataFrame(lst, columns=['objVariable1','objVariable2','numVariable','objValue1','objValue2', 'numValue1','numValue2', 'All', 'Good', 'Bad'])

            woetable['Share'] = woetable['All'] / woetable['All'].sum()
            woetable['Bad Rate'] = woetable['Bad'] / woetable['All']
            woetable['Distribution Good'] = (woetable['All'] - woetable['Bad']) / (woetable['All'].sum() - woetable['Bad'].sum())
            woetable['Distribution Bad'] = woetable['Bad'] / woetable['Bad'].sum()
            woetable['WoE'] = np.log(woetable['Distribution Good'] / woetable['Distribution Bad'])

            woetable = woetable.replace({'WoE': {np.inf: 0, -np.inf: 0}})

            woetable['IV'] = woetable['WoE'] * (woetable['Distribution Good'] - woetable['Distribution Bad'])

            woetable = woetable.sort_values(by=['numVariable','numValue1','numValue2'], ascending=[True,True,True])
            woetable.index = range(len(woetable.index))
            woetable = woetable.dropna()
            if iv>woetable['IV'].sum():
                continue
            else:
                ITOG=woetable#.values.tolist()
                iv = woetable['IV'].sum()
    else:
        for i in scale:
            val1=-99999 if i==scale[0] else val2# np.percentile(df[feature],k*i)
            val2=999999 if i==scale[len(scale)-1] else i
            lst.append(['None',
                        'None',
                        feature,
                        'None',# Variable
                        'None',
                        val1,                                                            # Value1
                        val2,
                        df[(df[feature]>=val1) & (df[feature] < val2)].count()[feature],                        # All
                        df[(df[feature]>=val1) & (df[feature] < val2) & (df['bg'] == 0)].count()[feature],  # Good (think: Fraud == 0)
                        df[(df[feature]>=val1) & (df[feature] < val2) & (df['bg'] == 1)].count()[feature]]) # Bad (think: Fraud == 1)

        woetable = pd.DataFrame(lst, columns=['objVariable1','objVariable2','numVariable','objValue1','objValue2', 'numValue1','numValue2', 'All', 'Good', 'Bad'])

        woetable['Share'] = woetable['All'] / woetable['All'].sum()
        woetable['Bad Rate'] = woetable['Bad'] / woetable['All']
        woetable['Distribution Good'] = (woetable['All'] - woetable['Bad']) / (woetable['All'].sum() - woetable['Bad'].sum())
        woetable['Distribution Bad'] = woetable['Bad'] / woetable['Bad'].sum()
        woetable['WoE'] = np.log(woetable['Distribution Good'] / woetable['Distribution Bad'])

        woetable = woetable.replace({'WoE': {np.inf: 0, -np.inf: 0}})

        woetable['IV'] = woetable['WoE'] * (woetable['Distribution Good'] - woetable['Distribution Bad'])

        woetable = woetable.sort_values(by=['numVariable','numValue1','numValue2'], ascending=[True,True,True])
        woetable.index = range(len(woetable.index))
        woetable = woetable.dropna()
        ITOG=woetable#.values.tolist()
        iv = woetable['IV'].sum()   
    # print(iv)

    return ITOG


def iv_numeric_object(df,object_feature,numeric_feature,num=20):
    """
    Set pr=True to enable printing of output.
    
    Output: 
      * iv: float,
      * woetable: pandas.woetableFrame
    """
    iv=0
    h=100/np.arange(2,num)
    for k in h:
        lst = []
        for nn in range(df[object_feature].nunique()):
            val = list(df[object_feature].unique())[nn]
            for i in np.arange(0,100//k):
                val1=-99999 if i==np.arange(0,100//k)[0] else val2 #np.percentile(df[df[object_feature]==val][numeric_feature],k*i)
                val2=999999 if i==np.arange(0,100//k)[len(np.arange(0,100//k))-1] else np.percentile(df[numeric_feature],k*(i+1))
                lst.append([object_feature,
                            'None',
                            numeric_feature, 
                            val,# Variable
                            'None',
                            val1,                                                            # Value1
                            val2,
                            df[(df[object_feature] == val) &(df[numeric_feature]>=val1) & (df[numeric_feature] < val2)].count()[numeric_feature],                        # All
                            df[(df[object_feature] == val) & (df[numeric_feature]>=val1) & (df[numeric_feature] < val2) & (df['bg'] == 0)].count()[numeric_feature],  # Good (think: Fraud == 0)
                            df[(df[object_feature] == val) & (df[numeric_feature]>=val1) & (df[numeric_feature] < val2) & (df['bg'] == 1)].count()[numeric_feature]]) # Bad (think: Fraud == 1)

        woetable = pd.DataFrame(lst, columns=['objVariable1','objVariable2','numVariable','objValue1','objValue2', 'numValue1','numValue2', 'All', 'Good', 'Bad'])

        woetable['Share'] = woetable['All'] / woetable['All'].sum()
        woetable['Bad Rate'] = woetable['Bad'] / woetable['All']
        woetable['Distribution Good'] = (woetable['All'] - woetable['Bad']) / (woetable['All'].sum() - woetable['Bad'].sum())
        woetable['Distribution Bad'] = woetable['Bad'] / woetable['Bad'].sum()
        woetable['WoE'] = np.log(woetable['Distribution Good'] / woetable['Distribution Bad'])

        woetable = woetable.replace({'WoE': {np.inf: 0, -np.inf: 0}})

        woetable['IV'] = woetable['WoE'] * (woetable['Distribution Good'] - woetable['Distribution Bad'])

        woetable = woetable.sort_values(by=['objVariable1','numVariable','objValue1', 'numValue1','numValue2'], ascending=[True,True,True,True,True])
        woetable.index = range(len(woetable.index))
        woetable = woetable.dropna()
        if iv > woetable['IV'].sum():
            continue
        else:
            ITOG=woetable#.values.tolist()
            iv = woetable['IV'].sum()
    # print(iv)

    return ITOG

def ks(y,pred):
    main=pd.DataFrame(scipy.vstack([y,pred]).T,columns=['bg','score'],)
    main=main.sort_values(['score'],ascending=False)
    g=main['bg'].replace({1:0,0:1}).cumsum()/main.loc[main['bg']==0]['bg'].count()
    b=main['bg'].cumsum()/main.loc[main['bg']==1]['bg'].count()
    k=abs(g.reset_index(drop=True).quantile([0.02,0.14,0.26,0.38,0.5,0.62,0.74,0.86,0.98,1])-b.reset_index(drop=True).quantile([0.02,0.14,0.26,0.38,0.5,0.62,0.74,0.86,0.98,1])).max()
    return k

def gini(actual,pred,weight=None):
    pdf= pd.DataFrame(scipy.vstack([actual,pred]).T,columns=['Actual','Predicted'],)
    pdf= pdf.sort_values('Predicted')
    if weight is None:
        pdf['Weight'] = 1.0
    pdf['CummulativeWeight'] = np.cumsum(pdf['Weight'])
    pdf['CummulativeWeightedActual'] = np.cumsum(pdf['Actual']*pdf['Weight'])
    TotalWeight = sum(pdf['Weight'])
    Numerator = sum(pdf['CummulativeWeightedActual']*pdf['Weight'])
    Denominator = sum(pdf['Actual']*pdf['Weight']*TotalWeight)
    Gini = 1.0 - 2.0 * Numerator/Denominator
    return Gini 
def iv_special(df,numeric_feature,special_feature,num=20):
    """
    Set pr=True to enable printing of output.
    
    Output: 
      * iv: float,
      * woetable: pandas.woetableFrame
    """
    iv=0
    h=100/np.arange(2,num)
    for k in h:
        lst = []
        for nn in range(0,2):
            for i in np.arange(0,100//k):
                val1=-99999 if i==np.arange(0,100//k)[0] else val2 #np.percentile(df[df[object_feature]==val][numeric_feature],k*i)
                val2=999999 if i==np.arange(0,100//k)[len(np.arange(0,100//k))-1] else np.percentile(df[numeric_feature],k*(i+1))
                start_val=nn
                if nn==0:
                    val = 1 if val1<6 else 2
                    start_val=0
                else:
                    val = 99999
                    start_val=1 if val1<6 else 2
                lst.append([special_feature,
                            'None',
                            numeric_feature, 
                            start_val,# Variable
                            val,
                            val1,                                                            # Value1
                            val2,
                            df[(df[special_feature] >=start_val)&(df[special_feature]<val) &(df[numeric_feature]>=val1) & (df[numeric_feature] < val2)].count()[numeric_feature],                        # All
                            df[(df[special_feature] >=start_val)&(df[special_feature]<val) & (df[numeric_feature]>=val1) & (df[numeric_feature] < val2) & (df['bg'] == 0)].count()[numeric_feature],  # Good (think: Fraud == 0)
                            df[(df[special_feature] >=start_val)&(df[special_feature]<val) & (df[numeric_feature]>=val1) & (df[numeric_feature] < val2) & (df['bg'] == 1)].count()[numeric_feature]]) # Bad (think: Fraud == 1)

        woetable = pd.DataFrame(lst, columns=['objVariable1','objVariable2','numVariable','objValue1','objValue2', 'numValue1','numValue2', 'All', 'Good', 'Bad'])

        woetable['Share'] = woetable['All'] / woetable['All'].sum()
        woetable['Bad Rate'] = woetable['Bad'] / woetable['All']
        woetable['Distribution Good'] = (woetable['All'] - woetable['Bad']) / (woetable['All'].sum() - woetable['Bad'].sum())
        woetable['Distribution Bad'] = woetable['Bad'] / woetable['Bad'].sum()
        woetable['WoE'] = np.log(woetable['Distribution Good'] / woetable['Distribution Bad'])

        woetable = woetable.replace({'WoE': {np.inf: 0, -np.inf: 0}})

        woetable['IV'] = woetable['WoE'] * (woetable['Distribution Good'] - woetable['Distribution Bad'])

        woetable = woetable.sort_values(by=['objVariable1','numVariable','objValue1', 'numValue1','numValue2'], ascending=[True,True,True,True,True])
        woetable.index = range(len(woetable.index))
        woetable = woetable.dropna()
        if iv > woetable['IV'].sum():
            continue
        else:
            ITOG=woetable#.values.tolist()
            iv = woetable['IV'].sum()
    # print(iv)
    ITOG=ITOG.reset_index(drop=True)
    ITOG2=pd.DataFrame(columns=ITOG.columns)
    for m in np.arange(0,int(len(ITOG)/2)):
        if m<len(ITOG):
            ITOG2.loc[len(ITOG2)]=ITOG.loc[m]
            ITOG2.loc[len(ITOG2)]=ITOG.loc[m+int(len(ITOG)/2)]
        else:
            break

    return ITOG2